Here are some models I tried to run (many have these were tuned various different ways) before finding the model with the right hyperparameters providing the high accuracy metrics and minimal to no overfitting.

In [1]:
#For Google Collab
from google.colab import drive, auth
from googleapiclient.discovery import build
#For data manipulation
import numpy as np 
import pandas as pd 
import random
#For Plotting
import matplotlib.pyplot as plt
#For Modeling
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers, regularizers

Using TensorFlow backend.


In [2]:
# Load the Drive helper and mount
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
auth.authenticate_user()

In [0]:
drive_service = build('drive', 'v3')

In [0]:
#Create variable for image and batch size along with number of epochs committing to for model
height = 224
width = 224
batch_size = 32

train = ImageDataGenerator(rescale = 1/.255,rotation_range=30,horizontal_flip=True)
test = ImageDataGenerator(rescale=1/.255,rotation_range=30,horizontal_flip=True)

In [6]:
#Seting up train, test, and val sets for model

train_set = train.flow_from_directory('../content/drive/My Drive/Chest_X_rays/All_Chest/train'
                                      ,target_size=(width,height), 
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                     seed = 122)
test_set = train.flow_from_directory('../content/drive/My Drive/Chest_X_rays/All_Chest/test'
                                      ,target_size=(width,height), 
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                    seed = 122)
val_set = train.flow_from_directory('../content/drive/My Drive/Chest_X_rays/All_Chest/val'
                                      ,target_size=(width,height), 
                                      batch_size=batch_size,
                                      class_mode='categorical',                                   
                                    seed = 122)

Found 4747 images belonging to 3 classes.
Found 1885 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [7]:
#Setting up model  94.13 %
#Model
model = Sequential()

#Input layer 1
model.add(Conv2D(32,(3,3),input_shape=(height,width,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

#Hidden layer 2
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#Hidden layer 3
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

#Hidden layer 4
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))

#Hidden layer 5
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))

#Hidden layer 6
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())

#Output layer
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=3,activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
batch_size = 32
epochs = 10

adam = optimizers.adam(lr=0.0001) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=1, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.00000000001)

history = model.fit_generator(train_set, steps_per_epoch=25, epochs=epochs, validation_steps=50, verbose=1, validation_data=test_set, callbacks = [rt])
history

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
24/25 [===========================>..] - ETA: 12s - loss: 1.4339 - acc: 0.7070

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Setting up model  94.13 % ---- Try decreasing the regulizer and dropout values
#Model
model = Sequential()

#Input layer 1
model.add(Conv2D(128,(3,3),input_shape=(height,width,3),kernel_regularizer=regularizers.l2(0.3),activation='relu'))
model.add(Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#Hidden layer 2
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.6))

#Hidden layer 3
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.7))

#Hidden layer 4
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.6))


#Hidden layer 5
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.5),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.5))

#Output layer
model.add(Flatten())
model.add(Dense(units=3,activation='softmax'))

In [0]:
batch_size = 32
epochs = 10

adam = optimizers.adam(lr=0.001) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=1, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.00000000001)

history = model.fit_generator(train_set, steps_per_epoch=5, epochs=epochs, validation_steps=50, verbose=1, validation_data=test_set, callbacks=[rt])
history

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Setting up model  94.13 % ---- Try decreasing the regulizer and dropout values
#Model
model = Sequential()

#Input layer 1
model.add(Conv2D(64,(3,3),input_shape=(height,width,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

#Hidden layer 2
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))


#Hidden layer 4
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))


#Hidden layer 5
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.2),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))


#Output layer
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(units=3,activation='softmax'))

In [0]:
batch_size = 32
epochs = 10

adam = optimizers.adam(lr=0.1) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.00000001)

history = model.fit_generator(train_set, steps_per_epoch=5, epochs=epochs, validation_steps=50, verbose=1, validation_data=test_set, callbacks=[rt])
history

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Setting up model  increasing and adding dropouts to help regularization
#Model
model = Sequential()

#Input layer 1
model.add(Conv2D(128,(3,3),input_shape=(height,width,3),activation='relu'))
model.add(Conv2D(28,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 2
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 3
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 4
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 5
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.2))

#Hidden layer 6
model.add(Conv2D(16,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

#Output layer
model.add(Flatten())
model.add(Dense(units=32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=3,activation='softmax'))

In [0]:
batch_size = 32
epochs = 10

adam = optimizers.adam(lr=0.00000001) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_test2 = model.fit_generator(train_set, steps_per_epoch=25, epochs=epochs, validation_steps=50, verbose=1, validation_data=test_set)
history_test2

In [0]:
plt.plot(history_test2.history['loss'], label = 'Train')
plt.plot(history_test2.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history_test2.history['acc'], label = 'Train')
plt.plot(history_test2.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
batch_size = 32
epochs = 10

adam = optimizers.adam(lr=0.1) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.00000001)

history = model.fit_generator(train_set, steps_per_epoch=5, epochs=epochs, validation_steps=10, verbose=1, validation_data=test_set, callbacks=[rt])
history

In [0]:
plt.plot(history_test2.history['loss'], label = 'Train')
plt.plot(history_test2.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history_test2.history['acc'], label = 'Train')
plt.plot(history_test2.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Setting up model  adding regularization
#Model
model = Sequential()
​
#Input layer 1
model.add(Conv2D(256,(3,3),input_shape=(height,width,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(Conv2D(256,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
​
#Hidden layer 2
model.add(Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
​
#Hidden layer 3
model.add(Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(Conv2D(128,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
​
#Hidden layer 4
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
​
#Hidden layer 5
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Dropout(0.2))
​
#Hidden layer 6
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
​
#Output layer
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(units=3,activation='softmax'))

In [0]:
batch_size = 32
epochs = 10

adam = optimizers.adam(lr=0.1) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.00000001)

history = model.fit_generator(train_set, steps_per_epoch=5, epochs=epochs, validation_steps=10, verbose=1, validation_data=test_set, callbacks=[rt])
history

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Setting up model  adding regularization
#Model
model = Sequential()

#Input layer 1
model.add(Conv2D(128,(3,3),input_shape=(height,width,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
#model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 2
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
#model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 3
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
#model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 4
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
#model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 5
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
#model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Hidden layer 6
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

#Output layer
model.add(Flatten())
model.add(Dense(units=32,kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=3,activation='softmax'))

In [0]:
batch_size = 64
epochs = 10

adam = optimizers.adam(lr=0.1) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

rt=ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=1, min_lr=.00000001)

history = model.fit_generator(train_set, steps_per_epoch=5, epochs=epochs, validation_steps=5, verbose=1, validation_data=test_set, callbacks=[rt])
history

In [0]:
plt.plot(history.history['loss'], label = 'Train')
plt.plot(history.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history.history['acc'], label = 'Train')
plt.plot(history.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()

In [0]:
#Setting up model  adding regularization
#Model
model = Sequential()

#Input layer 1
model.add(Conv2D(128,(3,3),input_shape=(height,width,3),kernel_regularizer=regularizers.l2(0.1),activation='relu'))
#model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#Hidden layer 2
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.1),activation='relu'))
#model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#Hidden layer 3
model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.1),activation='relu'))
#model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#Hidden layer 4
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.1),activation='relu'))
#model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#Hidden layer 5
model.add(Conv2D(32,(3,3),kernel_regularizer=regularizers.l2(0.1),activation='relu'))
#model.add(Conv2D(64,(3,3),kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#Output layer
model.add(Flatten())
model.add(Dense(units=3,activation='softmax'))

In [0]:
batch_size = 16
epochs = 5

adam = optimizers.adam(lr=0.0001) #, decay=1e-6, momentum=0.9, nesterov=True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_test5 = model.fit_generator(train_set, steps_per_epoch=5, epochs=epochs, validation_steps=5, verbose=1, validation_data=test_set)
history_test5

In [0]:
plt.plot(history_test5.history['loss'], label = 'Train')
plt.plot(history_test5.history['val_loss'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss')
plt.show()

plt.plot(history_test5.history['acc'], label = 'Train')
plt.plot(history_test5.history['val_acc'], label = 'Test')
plt.legend(loc = 'upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy')
plt.show()